In [1]:
from pennylane import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pennylane as qml
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [10]:
def powerFaultData():

    powerData = pd.read_csv('Testdata.csv')
    
    X = []
    Y = []

    for i in powerData.iterrows():
        X.append([i[1][p] for p in range(0, len(i[1]))])
        Y.append(int(i[1][6]))

    Y = np.asarray(Y).astype(int)

    xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.95, shuffle=True, random_state=42)
    xTrain = preprocessing.normalize(xTrain)
    xTest = preprocessing.normalize(xTest)

    return xTrain, yTrain

In [11]:
def target_alignment(
    X,
    Y,
    kernel,
    assume_normalized_kernel=False,
    rescale_class_labels=True,
):
    """Kernel-target alignment between kernel and labels."""

    K = qml.kernels.square_kernel_matrix(
        X,
        kernel,
        assume_normalized_kernel=assume_normalized_kernel,
    )

    if rescale_class_labels:
        nplus = np.count_nonzero(np.array(Y) == 1)
        nminus = len(Y) - nplus
        _Y = np.array([y / nplus if y == 1 else y / nminus for y in Y])
    else:
        _Y = np.array(Y)

    T = np.outer(_Y, _Y)
    inner_product = np.sum(K * T)
    norm = np.sqrt(np.sum(K * K) * np.sum(T * T))
    inner_product = inner_product / norm

    return inner_product

In [12]:
dev = qml.device("default.qubit", wires=6, shots=None)
wires = dev.wires.tolist()

def layer(x, params, wires, i0=0, inc=1):
    """Building block of the embedding ansatz"""
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)], wires=[wire])
        #qml.RZ(x[i], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])
    
def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))
        
adjoint_ansatz = qml.adjoint(ansatz)

def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    return np.random.uniform(0, 2 * np.pi, (num_layers, 2, num_wires), requires_grad=True)

@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]

def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [13]:
X, Y = powerFaultData()

print("Data: ", X[0])
print("Label: ", Y[0])

Data:  [-8.12146716e-01  4.72210474e-01  3.42651867e-01 -1.82818371e-04
  8.01413195e-05  1.02677050e-04  4.96261294e-03]
Label:  4


/tmp/ipykernel_749/3204019547.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X.append([i[1][p] for p in range(0, len(i[1]))])
/tmp/ipykernel_749/3204019547.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y.append(int(i[1][6]))


In [14]:
init_params = random_params(num_wires=6, num_layers=6)
print(init_params)
kernel_value = kernel(X[0], X[1], init_params)
print(f"The kernel value between the first and second datapoint is {kernel_value:.3f}")
print('---------------------------------------------------------------------------------------------')
print('Quantum Circuit: ')
drawer = qml.draw(kernel_circuit)
print(drawer(X[0], X[1], init_params))
print('---------------------------------------------------------------------------------------------')

[[[3.01385139e+00 2.35969983e+00 4.52578167e+00 8.53020798e-01
   1.19297017e+00 2.29111652e+00]
  [2.89090414e+00 6.52201892e-02 5.76761688e+00 1.05877495e+00
   2.96540138e+00 4.68253659e+00]]

 [[2.58139110e+00 6.09044173e-01 7.17189490e-01 2.43707266e+00
   3.03328309e+00 5.64398681e+00]
  [1.54051218e+00 4.51072653e-01 6.94431746e-01 3.94434554e+00
   3.97574858e+00 3.21199038e+00]]

 [[4.35173300e+00 5.47767362e+00 1.10308304e+00 6.05300700e+00
   1.81972783e+00 3.40962751e-01]
  [3.97288621e-03 4.03467976e+00 1.55113062e+00 2.98021553e+00
   3.83874411e+00 6.25788899e-01]]

 [[3.29850153e+00 3.01222041e+00 3.75264056e+00 2.97602474e+00
   2.12968467e+00 6.63706083e-01]
  [1.34872418e+00 2.49553445e+00 2.00790714e+00 1.16075556e+00
   3.52805903e+00 3.40025332e+00]]

 [[6.04012764e+00 1.85519446e+00 4.38200109e+00 4.21189848e+00
   3.34229170e+00 3.26087790e+00]
  [8.33462374e-01 3.43371926e+00 1.78281675e+00 5.19793725e+00
   4.41199819e+00 1.94662236e+00]]

 [[6.15329605e-01 2.

In [15]:
print("Kernel Alignment with Gradient Descent")

params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(500):
    # Choose subset of datapoints to compute the KTA on.
    
    subset = np.random.choice(list(range(len(X))), 4)

    #print(type(subset))
    #print(type(Y))
    
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")


trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

Kernel Alignment with Gradient Descent
Step 10 - Alignment = 0.649
Step 20 - Alignment = 0.659
Step 30 - Alignment = 0.680
Step 40 - Alignment = 0.702
Step 50 - Alignment = 0.722
Step 60 - Alignment = 0.741
Step 70 - Alignment = 0.768
Step 80 - Alignment = 0.790
Step 90 - Alignment = 0.805
Step 100 - Alignment = 0.822
Step 110 - Alignment = 0.832
Step 120 - Alignment = 0.841
Step 130 - Alignment = 0.849
Step 140 - Alignment = 0.854
Step 150 - Alignment = 0.861
Step 160 - Alignment = 0.866
Step 170 - Alignment = 0.869
Step 180 - Alignment = 0.873
Step 190 - Alignment = 0.876
Step 200 - Alignment = 0.879
Step 210 - Alignment = 0.882
Step 220 - Alignment = 0.885
Step 230 - Alignment = 0.887
Step 240 - Alignment = 0.889
Step 250 - Alignment = 0.890
Step 260 - Alignment = 0.891
Step 270 - Alignment = 0.893
Step 280 - Alignment = 0.894
Step 290 - Alignment = 0.895
Step 300 - Alignment = 0.896
Step 310 - Alignment = 0.897
Step 320 - Alignment = 0.898
Step 330 - Alignment = 0.899
Step 340 - Al

In [ ]:
print("Kernel Alignment with Gradient Descent with Active Learning")

params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(500):
    # Choose subset of datapoints to compute the KTA on.
    
    subset = np.random.choice(list(range(len(X))), 4) 

    #print(type(subset))
    #print(type(Y))
    
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")